[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Training_Tutorial.ipynb)

## Create the models
Import the models from the ISR package and create

- a RRDN super scaling network
- a discriminator network for GANs training
- a VGG19 feature extractor to train with a perceptual loss function

Carefully select
- 'x': this is the upscaling factor (2 by default)
- 'layers_to_extract': these are the layers from the VGG19 that will be used in the perceptual loss (leave the default if you're not familiar with it)
- 'lr_patch_size': this is the size of the patches that will be extracted from the LR images and fed to the ISR network during training time

Play around with the other architecture parameters

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
# import tensorflow as tf
# # 查看gpu和cpu的数量
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
# print("Num GPUs Available: ", len((tf.config.experimental.list_physical_devices('GPU'))))
# print(gpus, cpus)

In [3]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

In [4]:

# import torch
# print(torch.__version__)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #返回cuda表示成功
# #或者
# print(torch.cuda.is_available())
# #返回True表示成功

In [5]:
print(os.getcwd())

d:\Master_Thesis\image-super-resolution-master\notebooks


In [6]:
lr_train_patch_size = 40#40
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

In [7]:
print(os.getcwd())

d:\Master_Thesis\image-super-resolution-master\notebooks


## Give the models to the Trainer
The Trainer object will combine the networks, manage your training data and keep you up-to-date with the training progress through Tensorboard and the command line.

Here we do not use  the pixel-wise MSE but only the perceptual loss by specifying the respective weights in `loss_weights`

In [8]:
from ISR.train import Trainer
loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

path = os.getcwd()#d:/Master_Thesis/image-super-resolution-master/notebooks
log_dirs = {'logs': path + '/ISR_chr/logs', 'weights': path + '/ISR_chr/weights'}

learning_rate = {'initial_value': 0.00, 'decay_factor': 0.5, 'decay_frequency': 30}
#learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}
flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    # lr_train_dir=path + '/ISR_chr/data/train_LR/',
    # hr_train_dir=path + '/ISR_chr/data/train_HR_24/',
    # lr_valid_dir=path + '/ISR_chr/data/valid_LR/',
    # hr_valid_dir=path + '/ISR_chr/data/valid_HR_2/',
    # lr_train_dir=path + '/ISR_chr/data/final_data/data_aug/train/',
    # hr_train_dir=path + '/ISR_chr/data/final_data/labels_aug/train/',
    # lr_valid_dir=path + '/ISR_chr/data/final_data/data_aug/val/',
    # hr_valid_dir=path + '/ISR_chr/data/final_data/labels_aug/val/',
    lr_train_dir=path + '/ISR_chr/data/final_data/data_aug_resized/train/',
    hr_train_dir=path + '/ISR_chr/data/final_data/labels_aug_resized/train/',
    lr_valid_dir=path + '/ISR_chr/data/final_data/data_resized/val/',
    hr_valid_dir=path + '/ISR_chr/data/final_data/labels_resized/val/',
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2k',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=33,#33,#40
)


Choose epoch number, steps and batch size and start training

In [9]:
trainer.train(
    epochs=300,#10
    steps_per_epoch=20,#20
    batch_size=4,#4
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)


Training details:
  training_parameters: 
    lr_train_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/final_data/data_aug_resized/train/
    hr_train_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/final_data/labels_aug_resized/train/
    lr_valid_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/final_data/data_resized/val/
    hr_valid_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/final_data/labels_resized/val/
    loss_weights: {'generator': 0.0, 'feature_extractor': 0.0833, 'discriminator': 0.01}
    log_dirs: {'logs': 'd:\\Master_Thesis\\image-super-resolution-master\\notebooks/ISR_chr/logs', 'weights': 'd:\\Master_Thesis\\image-super-resolution-master\\notebooks/ISR_chr/weights'}
    fallback_save_every_n_epochs: 2
    dataname: div2k
    n_validation: 33
    flatness: {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}
    learning_rate: {'initial_value'

Epoch 0/300
Current learning rate: 0.0020000000949949026
100%|██████████| 20/20 [02:16<00:00,  6.85s/it]
Epoch 0 took      137.0s


132/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.7748949554833499, 'val_generator_loss': 0.45703274, 'val_discriminator_loss': 1.0202717, 'val_feature_extractor_loss': 2.626825, 'val_feature_extractor_1_loss': 15.73313, 'val_generator_PSNR_Y': 7.3650513, 'train_d_real_loss': 0.030985234, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.032868896, 'train_d_fake_accuracy': 0.999375, 'train_loss': 0.8190871, 'train_generator_loss': 0.5357409, 'train_discriminator_loss': 0.457077, 'train_feature_extractor_loss': 2.4864838, 'train_feature_extractor_1_loss': 17.069729, 'train_generator_PSNR_Y': 6.5242977}
val_generator_PSNR_Y improved from       -inf to    7.36505
Saving weights
Epoch 1/300
Current learning rate: 0.0020000000949949026
100%|██████████| 20/20 [02:18<00:00,  6.91s/it]
Epoch 1 took      138.3s


132/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

132/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': nan, 'val_generator_loss': nan, 'val_discriminator_loss': nan, 'val_feature_extractor_loss': nan, 'val_feature_extractor_1_loss': nan, 'val_generator_PSNR_Y': nan, 'train_d_real_loss': nan, 'train_d_real_accuracy': 0.0, 'train_d_fake_loss': nan, 'train_d_fake_accuracy': 1.0, 'train_loss': nan, 'train_generator_loss': nan, 'train_discriminator_loss': nan, 'train_feature_extractor_loss': nan, 'train_feature_extractor_1_loss': nan, 'train_generator_PSNR_Y': nan}
val_generator_PSNR_Y did not improve.
Saving weights anyways.
Epoch 186/300
Current learning rate: 3.125000148429535e-05
Current flatness treshold: 0.15
100%|██████████| 20/20 [02:22<00:00,  7.14s/it]
Epoch 186 took      142.9s


132/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': nan, 'val_generator_loss': nan, 'val_discriminator_loss': nan, 'val_feature_extractor_loss': nan, 'val_feature_extractor_1_loss': nan, 'val_generator_PSNR_Y': nan, 'train_d_real_loss': nan, 'train_d_real_accuracy': 0.0, 'train_d_fake_loss': nan, 'train_d_fake_accuracy': 1.0, 'train_loss': nan, 'train_generator_loss': nan, 'train_discriminator_loss': nan, 'train_feature_extractor_loss': nan, 'train_feature_extractor_1_loss': nan, 'train_generator_PSNR_Y': nan}
val_generator_PSNR_Y did not improve.
Epoch 187/300
Current learning rate: 3.125000148429535e-05
Current flatness treshold: 0.15
 70%|███████   | 14/20 [01:43<00:44,  7.41s/it]


KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%load_ext tensorboard